In [1]:
import math
import torch
import torch.nn.functional as F
import tenseal as ts
import numpy as np
from skimage.util.shape import view_as_windows

### Prepaire The Data

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms

# how many samples per batch to load
batch_size = 20

# convert data to a normalized torch.FloatTensor
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# choose the training and test datasets
train_data = datasets.MNIST('data', train=True,
                              download=True, transform=transform)
test_data = datasets.MNIST('data', train=False,
                             download=True, transform=transform)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

### Define the Network Architecture

In [3]:
# define the NN architecture
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()        
        # convolutional layer (sees 28x28 image tensor)
        self.conv1 = torch.nn.Conv2d(1, 4, kernel_size=7, padding=0, stride=3)
        # 4 * 8 * 8 == 256
        # linear layer (256 -> 64)
        self.fc1 = torch.nn.Linear(256, 64)
        # linear layer (64 -> 10)
        self.fc2 = torch.nn.Linear(64, 10)

    def forward(self, x):
        # conv layer
        x = self.conv1(x)        
        # squaring
        x = x * x
        # flatten image input
        x = x.view(-1, 256)
        # add hidden layer, with square activation function
        x = self.fc1(x)
        # squaring
        x = x * x
        # output layer
        x = self.fc2(x)
        return x

# initialize the NN
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 4, kernel_size=(7, 7), stride=(3, 3))
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)


###  Specify Loss Function

It's recommended that you use cross-entropy loss for classification. If you look at the documentation (linked above), you can see that PyTorch's cross entropy function applies a softmax funtion to the output layer *and* then calculates the log loss.

In [4]:
# specify loss function (categorical cross-entropy)
criterion = torch.nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

### Train the Network


In [5]:
# number of epochs to train the model
n_epochs = 5

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
    
    # save them model
    torch.save(model.state_dict(), 'model_mnist_tenseal.pt')

Epoch: 1 	Training Loss: 5.657106
Epoch: 2 	Training Loss: 2.330808
Epoch: 3 	Training Loss: 1.785717
Epoch: 4 	Training Loss: 1.525375
Epoch: 5 	Training Loss: 1.298905


###  Load the Model

In [6]:
model.load_state_dict(torch.load('model_mnist_tenseal.pt'))

<All keys matched successfully>

### Test the Trained Network

In [7]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for evaluation

for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (str(i)))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.094291

Test Accuracy of     0: 99% (974/980)
Test Accuracy of     1: 98% (1121/1135)
Test Accuracy of     2: 98% (1017/1032)
Test Accuracy of     3: 97% (986/1010)
Test Accuracy of     4: 98% (965/982)
Test Accuracy of     5: 96% (857/892)
Test Accuracy of     6: 97% (934/958)
Test Accuracy of     7: 95% (979/1028)
Test Accuracy of     8: 98% (957/974)
Test Accuracy of     9: 95% (964/1009)

Test Accuracy (Overall): 97% (9754/10000)


### Encrypted Evaluation 

In this part, we will just focus on evaluating the CNN model with plain parameters on the encrypted test-set. We first create a PyTorch-like CNN model that can evaluate encrypted data.

In [8]:
# Create TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS, 8192*2, coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 40, 60]
)
# set the scale
context.global_scale = pow(2, 40)
# generated galois keys in order to do rotation on ciphertext vectors
context.generate_galois_keys()

In [9]:
class EncEvalModel:
    def __init__(self, torch_nn):
        self.conv1_out_channels = torch_nn.conv1.out_channels
        self.conv1_kernel_shape = torch_nn.conv1.kernel_size
        self.conv1_stride = torch_nn.conv1.stride[0]
        self.conv1_weight = torch_nn.conv1.weight.data.view(
            self.conv1_out_channels, self.conv1_kernel_shape[0],
            self.conv1_kernel_shape[1]
        ).tolist()
        self.conv1_bias = torch_nn.conv1.bias.data.tolist()
        
        self.fc1_weight = torch_nn.fc1.weight.T.data.tolist()
        self.fc1_bias = torch_nn.fc1.bias.data.tolist()
        
        self.fc2_weight = torch_nn.fc2.weight.T.data.tolist()
        self.fc2_bias = torch_nn.fc2.bias.data.tolist()
        
        
    def forward(self, x):
        # conv layer
        x_enc, windows_nb = ts.im2col_encoding(
            context, x.tolist(), self.conv1_kernel_shape[0],
            self.conv1_kernel_shape[1], self.conv1_stride
        )

        channels = []
        for kernel in self.conv1_weight:
            # print("windows number: ", windows_nb)
            y = x_enc.conv2d_im2col(kernel, windows_nb)
            channels.append(y)
        
        x = ts.pack_vectors(channels)
        
        # squaring
        x.square_()
        # no need to flat
        # fc1 layer
        x = x.mm(self.fc1_weight) + self.fc1_bias
        # squaring
        x.square_()
        # output layer
        x = x.mm(self.fc2_weight) + self.fc2_bias
        return x
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    
enc_eval_model = EncEvalModel(model)

In [10]:
%%time
from itertools import islice

# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)
for data, target in islice(test_loader, 100):
    # forward pass: compute predicted outputs by passing inputs to the model
    enc_output = enc_eval_model(data.view(28, 28).numpy())
    output = enc_output.decrypt()
    output = torch.tensor(output).view(1, -1)
    
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    # print(pred, correct)
    label = target.data[0]
    class_correct[label] += correct.item()
    class_total[label] += 1

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (str(i)))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of     0: 100% ( 8/ 8)
Test Accuracy of     1: 100% (10/10)
Test Accuracy of     2: 100% (10/10)
Test Accuracy of     3: 100% (11/11)
Test Accuracy of     4: 100% ( 9/ 9)
Test Accuracy of     5: 91% (11/12)
Test Accuracy of     6: 100% ( 6/ 6)
Test Accuracy of     7: 100% (13/13)
Test Accuracy of     8: 100% ( 9/ 9)
Test Accuracy of     9: 91% (11/12)

Test Accuracy (Overall): 98% (98/100)
CPU times: user 1h 7min 8s, sys: 5.99 s, total: 1h 7min 14s
Wall time: 11min 26s
